In [1]:
import sys
!{sys.executable} -m pip install wandb optuna "ray[tune]" -q

In [2]:
import os
os.chdir('/workspace')

In [3]:
!cd /workspace && git clone https://github.com/Eran-BA/PoT.git

Cloning into 'PoT'...
remote: Enumerating objects: 2185, done.
remote: Counting objects: 100% (196/196), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 2185 (delta 134), reused 126 (delta 69), pack-reused 1989 (from 2)
Receiving objects: 100% (2185/2185), 1.35 MiB | 17.50 MiB/s, done.
Resolving deltas: 100% (1324/1324), done.


In [ ]:
import wandb
import os

# Set the NEW key
os.environ["WANDB_API_KEY"] = ""

# Force re-login
wandb.login(key=os.environ["WANDB_API_KEY"], relogin=True, force=True)

# Test it
run = wandb.init(project="sudoku-hpo-test", name="test-run")
wandb.log({"test": 1})
wandb.finish()
print("✓ Success!")

wandb: ERROR Failed to detect the name of this notebook. You can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: eranbt92 (eranbt92-open-university-of-israel) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: ERROR The nbformat package was not found. It is required to save notebook history.


test,▁
test,1


✓ Success!


In [5]:

import os
import sys
import glob
import torch
import wandb
from datetime import datetime
# Download artifact
import wandb
run = wandb.init()
artifact = run.use_artifact('eranbt92-open-university-of-israel/sudoku-hpo/best-model-trial_lr5.95e-04_L4_asyncTrue:v1', type='model')
artifact_dir = artifact.download()
wandb.finish()

# Find the checkpoint file
checkpoint_path = os.path.join(artifact_dir, "trial_lr5.95e-04_L4_asyncTrue_best.pt")
if not os.path.exists(checkpoint_path):
    # List files to find the right one
    print("Files in artifact:")
    for f in os.listdir(artifact_dir):
        print(f"  {f}")
    checkpoint_path = os.path.join(artifact_dir, os.listdir(artifact_dir)[0])

# Load checkpoint
checkpoint = torch.load(checkpoint_path, map_location='cpu', weights_only=False)

print(f"✓ Loaded checkpoint:")
print(f"  Epoch: {checkpoint['epoch']}")
print(f"  Best Grid Acc: {checkpoint['best_grid_acc']:.2f}%")
print(f"  Config: {checkpoint['config']}")

wandb: Downloading large artifact 'best-model-trial_lr5.95e-04_L4_asyncTrue:v1', 271.08MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:08.3 (32.7MB/s)
wandb: ERROR The nbformat package was not found. It is required to save notebook history.


✓ Loaded checkpoint:
  Epoch: 200
  Best Grid Acc: 8.00%
  Config: {'lr': 0.000594874681321977, 'weight_decay': 0.2624967163228772, 'puzzle_lr_multiplier': 32.72290672827732, 'puzzle_weight_decay': 1.4293571265737601, 'H_cycles': 2, 'L_cycles': 4, 'halt_max_steps': 2, 'halt_exploration': 0.09275410183585496, 'dropout': 0.007625738023228556, 'beta2': 0.9106812512723371, 'warmup_steps': 2500, 'async_batch': True, 'batch_size': 768, 'epochs_per_trial': 2000, 'eval_interval': 100, 'checkpoint_interval': 100, 'checkpoint_dir': '/workspace/PoT/experiments/hpo_results/checkpoints', 'd_model': 512, 'n_heads': 8, 'H_layers': 2, 'L_layers': 2, 'd_ff': 2048, 'T': 4, 'hrm_grad_style': True, 'wandb_project': 'sudoku-hpo', 'wandb_group': 'sudoku_hpo_4xB200'}


In [7]:
from src.data import download_sudoku_dataset

# Download dataset
download_sudoku_dataset(
    output_dir='data/sudoku-extreme-10k-aug-100',
    subsample_size=10000,
    num_aug=100,
)

Note: Augmentation is now ON-THE-FLY (not pre-computed)


train.csv:   0%|          | 0.00/719M [00:00<?, ?B/s]

  Train puzzles: 9000, Val puzzles: 1000


Processing train: 100%|██████████| 9000/9000 [00:00<00:00, 71858.28it/s]


  train: 9000 puzzles (augmentation: on-the-fly)


Processing val: 100%|██████████| 1000/1000 [00:00<00:00, 70706.41it/s]

  val: 1000 puzzles (augmentation: on-the-fly)


test.csv:   0%|          | 0.00/79.4M [00:00<?, ?B/s]

Processing test: 422786it [00:06, 69787.80it/s]


  test: 422786 puzzles
✓ Dataset saved to data/sudoku-extreme-10k-aug-100


In [11]:
!pip install nbformat

In [ ]:
# ============================================================
# PHASE 2: Continue Training from W&B Artifact Checkpoint
# ============================================================
import os
import sys
import torch
import wandb
from datetime import datetime
import math

os.chdir('/workspace/PoT')
sys.path.insert(0, '/workspace/PoT')

# Config from checkpoint (already loaded)
config = checkpoint['config']
start_epoch = checkpoint['epoch']

print("=" * 60)
print("PHASE 2: Continue Training")
print("=" * 60)
print(f"Starting from epoch: {start_epoch}")
print(f"Best grid acc: {checkpoint['best_grid_acc']:.2f}%")

# ============================================================
# Setup Model and Data
# ============================================================
from torch.utils.data import DataLoader
from src.data import SudokuDataset
from src.pot.models import HybridPoHHRMSolver
from src.training import train_epoch, train_epoch_async, evaluate

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

# Load data
data_dir = 'data/sudoku-extreme-10k-aug-100'
train_dataset = SudokuDataset(data_dir, 'train')
val_dataset = SudokuDataset(data_dir, 'val')
test_dataset = SudokuDataset(data_dir, 'test')

batch_size = config['batch_size']
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

print(f"Data: {len(train_dataset)} train, {len(val_dataset)} val, {len(test_dataset)} test")

# Build model
model = HybridPoHHRMSolver(
    d_model=config['d_model'],
    n_heads=config['n_heads'],
    H_layers=config['H_layers'],
    L_layers=config['L_layers'],
    d_ff=config['d_ff'],
    dropout=config['dropout'],
    H_cycles=config['H_cycles'],
    L_cycles=config['L_cycles'],
    T=config['T'],
    num_puzzles=1,
    hrm_grad_style=config['hrm_grad_style'],
    halt_max_steps=config['halt_max_steps'],
    halt_exploration_prob=config['halt_exploration'],
).to(device)

# Load weights
model.load_state_dict(checkpoint['model_state_dict'])
print("✓ Model loaded")

# ============================================================
# Optimizers (lower LR for fine-tuning)
# ============================================================
finetune_lr = config['lr'] * 0.1  # 10x lower
total_epochs = 7000
eval_interval = 50

puzzle_lr = finetune_lr * config['puzzle_lr_multiplier']
betas = (0.9, config['beta2'])

puzzle_params = list(model.puzzle_emb.parameters())
model_params = [p for p in model.parameters() if p not in set(puzzle_params)]

optimizer = torch.optim.AdamW(model_params, lr=finetune_lr, weight_decay=config['weight_decay'], betas=betas)
puzzle_optimizer = torch.optim.AdamW(puzzle_params, lr=puzzle_lr, weight_decay=config['puzzle_weight_decay'], betas=betas)

# LR schedule
total_steps = total_epochs * len(train_loader)
warmup_steps = 500

def lr_lambda(step):
    if step < warmup_steps:
        return float(step) / float(max(1, warmup_steps))
    progress = float(step - warmup_steps) / float(max(1, total_steps - warmup_steps))
    return 0.1 + 0.9 * 0.5 * (1.0 + math.cos(math.pi * progress))

scheduler = torch.optim.lr_scheduler.LambdaLR(optimizer, lr_lambda)
puzzle_scheduler = torch.optim.lr_scheduler.LambdaLR(puzzle_optimizer, lr_lambda)

print(f"✓ Optimizer: lr={finetune_lr:.2e}")

# ============================================================
# W&B
# ============================================================
run = wandb.init(
    project="sudoku-finetune",
    name=f"phase2_{datetime.now().strftime('%Y%m%d_%H%M')}",
    config={**config, "phase": 2, "start_epoch": start_epoch, "finetune_lr": finetune_lr}
)
print(f"✓ W&B: {run.url}")

# ============================================================
# Training Loop
# ============================================================
use_async = config['async_batch']
best_grid_acc = checkpoint['best_grid_acc']
save_dir = 'experiments/hpo_results/finetune_checkpoints'
os.makedirs(save_dir, exist_ok=True)

print(f"\nStarting training: {total_epochs} epochs\n")

for epoch in range(1, total_epochs + 1):
    if use_async:
        train_metrics = train_epoch_async(
            model, train_loader, optimizer, puzzle_optimizer,
            device, epoch, use_poh=True,
            scheduler=scheduler, puzzle_scheduler=puzzle_scheduler,
        )
    else:
        train_metrics = train_epoch(
            model, train_loader, optimizer, puzzle_optimizer,
            device, epoch, use_poh=True,
            scheduler=scheduler, puzzle_scheduler=puzzle_scheduler,
            constraint_weight=0.0,
        )
    
    train_dataset.on_epoch_end()
    
    if epoch % eval_interval == 0 or epoch == 1:
        val_metrics = evaluate(model, val_loader, device, use_poh=True)
        
        print(f"  → Val: loss={val_metrics['loss']:.4f}, cell={val_metrics['cell_acc']:.1f}%, grid={val_metrics['grid_acc']:.1f}%")
        
        if val_metrics['grid_acc'] > best_grid_acc:
            best_grid_acc = val_metrics['grid_acc']
            
            # Save locally
            best_path = os.path.join(save_dir, 'best_model.pt')
            torch.save({
                'epoch': start_epoch + epoch,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'best_grid_acc': best_grid_acc,
                'config': config,
            }, best_path)
            print(f"  🏆 New best: {best_grid_acc:.2f}%")
            
            # Upload to W&B with "best" alias (always points to latest best)
            artifact = wandb.Artifact("sudoku-finetune-best", type="model")
            artifact.add_file(best_path)
            run.log_artifact(artifact, aliases=["best", "latest"])
            print(f"  ✓ Uploaded to W&B (alias: best)")
        
        wandb.log({
            'epoch': start_epoch + epoch,
            'train_loss': train_metrics['loss'],
            'train_cell_acc': train_metrics['cell_acc'],
            'val_loss': val_metrics['loss'],
            'val_cell_acc': val_metrics['cell_acc'],
            'val_grid_acc': val_metrics['grid_acc'],
            'best_grid_acc': best_grid_acc,
        })


PHASE 2: Continue Training
Starting from epoch: 200
Best grid acc: 8.00%
Device: cuda
[train] Loaded 9000 puzzles
  Augmentation: ON-THE-FLY
[val] Loaded 1000 puzzles
  Augmentation: OFF
[test] Loaded 422786 puzzles
  Augmentation: OFF
Data: 9000 train, 1000 val, 422786 test
✓ Model loaded
✓ Optimizer: lr=5.95e-05


best_grid_acc,▁
epoch,▁
train_cell_acc,▁
train_loss,▁
val_cell_acc,▁
val_grid_acc,▁
val_loss,▁
best_grid_acc,11.6
epoch,201
train_cell_acc,71.72456
train_loss,0.6624


✓ W&B: https://wandb.ai/eranbt92-open-university-of-israel/sudoku-finetune/runs/esl8m9i8

Starting training: 7000 epochs





Epoch 1 (async):   0%|          | 0/9216 [00:00<?, ?it/s]

Epoch 1 (async):   0%|          | 16/9216 [00:00<07:15, 21.13it/s]

Epoch 1 (async):   0%|          | 16/9216 [00:00<07:15, 21.13it/s, loss=0.6749, cell_acc=70.0%, grid_acc=2.3%]

Epoch 1 (async):   8%|▊         | 768/9216 [00:01<00:13, 614.60it/s, loss=0.6749, cell_acc=70.0%, grid_acc=2.3%]

Epoch 1 (async):   8%|▊         | 768/9216 [00:01<00:13, 614.60it/s, loss=0.6555, cell_acc=71.3%, grid_acc=3.1%]

Epoch 1 (async):   9%|▊         | 797/9216 [00:02<00:13, 614.60it/s, loss=0.6852, cell_acc=70.8%, grid_acc=2.7%]

Epoch 1 (async):  17%|█▋        | 1536/9216 [00:02<00:13, 567.80it/s, loss=0.6852, cell_acc=70.8%, grid_acc=2.7%]

Epoch 1 (async):  17%|█▋        | 1536/9216 [00:02<00:13, 567.80it/s, loss=0.6626, cell_acc=71.2%, grid_acc=3.1%]

Epoch 1 (async):  17%|█▋        | 1579/9216 [00:03<00:13, 567.80it/s, loss=0.6695, cell_acc=71.0%, grid_acc=2.9%]

Epoch 1 (async):  25%|██▌       | 2304/9216 [00:04<00:12, 554.67it/s, lo

  → Val: loss=0.5797, cell=74.6%, grid=11.5%
  🏆 New best: 11.50%
  ✓ Uploaded to W&B (alias: best)




Epoch 2 (async):   0%|          | 0/9216 [00:00<?, ?it/s]

Epoch 2 (async):   0%|          | 19/9216 [00:00<06:10, 24.79it/s]

Epoch 2 (async):   0%|          | 19/9216 [00:00<06:10, 24.79it/s, loss=0.6633, cell_acc=70.7%, grid_acc=2.7%]

Epoch 2 (async):   8%|▊         | 768/9216 [00:01<00:13, 611.42it/s, loss=0.6633, cell_acc=70.7%, grid_acc=2.7%]

Epoch 2 (async):   8%|▊         | 768/9216 [00:01<00:13, 611.42it/s, loss=0.6499, cell_acc=72.0%, grid_acc=4.1%]

Epoch 2 (async):   9%|▊         | 799/9216 [00:02<00:13, 611.42it/s, loss=0.6694, cell_acc=71.5%, grid_acc=3.4%]

Epoch 2 (async):  17%|█▋        | 1536/9216 [00:02<00:13, 566.98it/s, loss=0.6694, cell_acc=71.5%, grid_acc=3.4%]

Epoch 2 (async):  17%|█▋        | 1536/9216 [00:02<00:13, 566.98it/s, loss=0.6532, cell_acc=72.0%, grid_acc=4.4%]

Epoch 2 (async):  17%|█▋        | 1585/9216 [00:03<00:13, 566.98it/s, loss=0.6685, cell_acc=71.7%, grid_acc=4.1%]

Epoch 2 (async):  25%|██▌       | 2305/9216 [00:04<00:12, 554.20it/s, lo

In [ ]:
# ============================================================
# Final Test Evaluation + Upload Best Model to W&B
# ============================================================
print("\n" + "=" * 60)
print("Final Test Evaluation (422k puzzles)")
print("=" * 60)

best_path = os.path.join(save_dir, 'best_model.pt')
if os.path.exists(best_path):
    model.load_state_dict(torch.load(best_path, weights_only=False)['model_state_dict'])

test_metrics = evaluate(model, test_loader, device, use_poh=True)

print(f"\n🎯 TEST RESULTS:")
print(f"   Cell Accuracy: {test_metrics['cell_acc']:.2f}%")
print(f"   Grid Accuracy: {test_metrics['grid_acc']:.2f}%")

wandb.log({'test_cell_acc': test_metrics['cell_acc'], 'test_grid_acc': test_metrics['grid_acc']})

# Upload best model artifact to W&B
if os.path.exists(best_path):
    artifact = wandb.Artifact(
        name="sudoku-continues-best",
        type="model",
        description=f"Best continues model - grid_acc={best_grid_acc:.2f}%, test_grid_acc={test_metrics['grid_acc']:.2f}%"
    )
    artifact.add_file(best_path)
    wandb.log_artifact(artifact)
    print(f"✓ Best model uploaded to W&B")

wandb.finish()

print(f"\n✓ Done! Best model: {best_path}")

In [ ]:
print("hey")